## Aula 13 - APIs

In [ ]:
import requests

In [ ]:
url = 'https://swapi.co/api/people/'
response = requests.get(url)
response_json = response.json()
#print(response_json)
#print(response_json['name'])

In [ ]:
from tkinter import *
class TkUrlImage():
    def __init__(self, window, url, width = 400, height = 400):
        self.url = url
        self.window = window
        self.width = width
        self.height = height

        raw_data = requests.get(self.url, stream=True).raw.data
        im = Image.open(io.BytesIO(raw_data))
        self.image = ImageTk.PhotoImage(im)
        self.me = Label(self.window, image=self.image, width=self.width, height=self.height)
    
    def as_label(self):
        return self.me
        
    

In [ ]:
import requests
import io
from tkinter import *
from PIL import Image, ImageTk

class Meowner():
    def __init__(self):
        self.janela = Tk()
        self.janela.title('Meowner')
        self.janela.geometry('800x600')
        self.janela.tk_setPalette(background= '#7e7d80', foreground = '#ffFF00', activeBackground = 'blue')
        
        self.btn_update = Button(self.janela, text='OTRO GATO!', command=self.update_cat)
        self.btn_update.pack()
        
        self.update_cat()
        
        self.janela.mainloop()
        
        
    def update_cat(self):
        try:
            self.image.destroy()
        except:
            pass
        
        self.cat_img_url = self.get_random_cat()
        self.image = TkUrlImage(window=self.janela, url=self.cat_img_url, width=600, height=400).as_label()
        self.image.pack()

    def get_random_cat(self):
        url = 'https://aws.random.cat/meow'
        r = requests.get(url)
        r_json = r.json()
        return r_json['file']

m = Meowner()

In [ ]:
import requests
import io
from tkinter import *
from PIL import Image, ImageTk

class Meowner():
    def __init__(self):
        self.janela = Tk()
        self.janela.title('Meowner')
        self.janela.geometry('800x600')
        self.janela.tk_setPalette(background= '#7e7d80', foreground = '#ffFF00', activeBackground = 'blue')
        
        self.btn_update = Button(self.janela, text='OTRO GATO!', command=self.update_cat)
        self.btn_update.pack()
        
        self.update_cat()
        
        self.janela.mainloop()
        
        
    def update_cat(self):
        self.cat_img_url = self.get_random_cat()
        self.image = self.get_tkinter_image(self.cat_img_url)
        try:
            self.label.destroy()
        except:
            pass
        self.label = Label(self.janela, image=self.image, width=600, height=400)
        self.label.pack()
        
    def get_tkinter_image(self, img_url):
        raw_data = requests.get(img_url, stream=True).raw.data
        im = Image.open(io.BytesIO(raw_data))
        image = ImageTk.PhotoImage(im)
        return image

    def get_random_cat(self):
        url = 'https://aws.random.cat/meow'
        r = requests.get(url)
        r_json = r.json()
        return r_json['file']

m = Meowner()

## Gatoflix - Com API Requests

In [36]:
import requests
import csv
import io
from tkinter import *
from tkinter import messagebox
from PIL import Image, ImageTk

class Gatoflix():
    def __init__(self):
        self.base_filmes = self.get_movie_base()
        self.do_layout()

    def get_movie_base(self):
        base = {}
        with open('movies.csv', 'r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                titulo = row['title']
                base[titulo] = row
                
        return base
    
    def do_movie_detail(self, evt):
        widget = evt.widget # isso volta o listbox que foi clicado
        index = widget.curselection()
        titulo_filme = widget.get(index)
        
        omdb_info = self.fetch_omdb_detail(titulo_filme)
        if omdb_info is not None:
            self.update_labels(omdb_info)
    
    def fetch_omdb_detail(self, title):
        title_fix = title[:title.index('(')].strip()
        year = title[title.index('('):title.index(')')].strip()
        url = 'https://www.omdbapi.com/?apiKey=d92c22a&t={}&y={}'.format(title_fix.replace(' ', '+'), year)
        response = requests.get(url).json()
        if 'Error' in response:
            messagebox.showinfo("ERRO!", "Filme não encontrado no OmDB")
            return None
        
        return {
            'year': response['Year'],
            'plot': response['Plot'],
            'poster': response['Poster'],
            'awards': response['Awards'],
            'title': title
        }
        
    def update_labels(self, infos):
        self.label_movie_title['text'] = '{}'.format(infos['title'])
        self.label_year['text'] = 'Ano: {}'.format(infos['year'])
        self.label_awards['text'] = 'Prêmios: {}'.format(infos['awards'])
        self.set_poster(infos['poster'], self.frame_image)
        
    def set_poster(self, url, window):
        self.image = self.get_tkinter_image(url)
        try:
            self.label_img.destroy()
        except:
            pass
        self.label_img = Label(window, image=self.image, width=600, height=400)
        self.label_img.pack()
        
    def get_tkinter_image(self, img_url):
        raw_data = requests.get(img_url, stream=True).raw.data
        im = Image.open(io.BytesIO(raw_data))
        image = ImageTk.PhotoImage(im)
        return image
        
    
    def do_layout(self):
        self.janela = Tk()
        self.janela.title('GATOFLIX')
        self.janela.geometry('1024x768')
        
        lbl_programa = Label(self.janela, text = "Gatoflix", font=('arial', 40, 'bold'))
        lbl_programa.pack()
        
        frame_detalhe = Frame(self.janela)
        frame_detalhe.pack(side = TOP)
        
        self.list_box = Listbox(frame_detalhe, width = 45)
        self.list_box.bind('<<ListboxSelect>>', self.do_movie_detail)
        self.list_box.pack()
        
        self.info = Frame(self.janela)
        self.info.pack(side=LEFT)
        
        self.label_movie_title = Label(self.info, text = "Aqui vai o texto", font=('arial', 32, 'italic'))
        self.label_movie_title.pack(side=TOP)
                
        self.label_year = Label(self.info, text = "Ano: ----", font=('arial', 20, 'bold'))
        self.label_year.pack(side=TOP)
        
        self.label_awards = Label(self.info, text = "Prêmios: ----", font=('arial', 20, 'bold'))
        self.label_awards.pack(side=TOP)
        
        # Inserindo no final do list box o titulo de cada filme
        # da base carregada no início
        for filme in sorted(self.base_filmes.keys()):
            self.list_box.insert(END, filme)
            
        self.frame_image = Frame(self.janela)
        self.frame_image.pack(side=LEFT)
            
        url = 'https://m.media-amazon.com/images/M/MV5BMDU2ZWJlMjktMTRhMy00ZTA5LWEzNDgtYmNmZTEwZTViZWJkXkEyXkFqcGdeQXVyNDQ2OTk4MzI@._V1_SX300.jpg'
        self.set_poster(url, self.frame_image)
        
        self.janela.mainloop()

In [37]:
gatoflix = Gatoflix()

## Exercício 4 - APIs

Para os próximos exercícios vamos utilizar uma API do Governo Federal para analisar os gastos por meio de cartão de 
pagamento. O link da API é:“http://www.transparencia.gov.br/api-de-dados/cartoes?mesExtratoInicio={}%2F{}&mesExtratoFim={}%2F{}&pagina={}”.format(mes_ini, ano_ini, mes_fim, ano_fim, pagina)onde os dados retornados correspondem a um intervalo definido por um mês e ano inicial (mes_ini e ano_ini) e um mês e ano final (mes_fim, ano_fim) e são apresentados 14 dados por página, com a primeira página sendo definida por pagina = 1.

In [10]:
import requests

mes_ini = '06'
ano_ini = 2018
mes_fim = '07'
ano_fim = 2018
pagina = 2
url = 'http://www.transparencia.gov.br/api-de-dados/cartoes?mesExtratoInicio={}%2F{}&mesExtratoFim={}%2F{}&pagina={}'.format(mes_ini, ano_ini, mes_fim, ano_fim, pagina)

response = requests.get(url)
print(response)

<Response [200]>


In [11]:
response.json()

[{'id': 141433432,
  'mesExtrato': '06/2018',
  'dataTransacao': '26/04/2018',
  'valorTransacao': '52,00',
  'tipoCartao': {'id': 1,
   'codigo': '1',
   'descricao': 'Cartão de Pagamento do Governo Federal - CPGF'},
  'estabelecimento': {'numeroInscricaoSocial': '',
   'nome': 'SO TINTAS TERESOPOLIS LIMITADA',
   'razaoSocialReceita': 'SO TINTAS TERESOPOLIS LIMITADA',
   'nomeFantasiaReceita': 'SO TINTAS',
   'cnae': {'codigoSecao': 'G',
    'secao': 'COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E MOTOCICLETAS',
    'codigoSubclasse': '4744099',
    'subclasse': 'Comércio varejista de materiais de construção em geral',
    'codigoDivisao': '47',
    'divisao': 'COMÉRCIO VAREJISTA',
    'codigoGrupo': '474',
    'grupo': 'Comércio varejista de material de construção',
    'codigoClasse': '47440',
    'classe': 'Comércio varejista de ferragens, madeira e materiais de construção'},
   'municipio': {'codigoIBGE': '3304557',
    'nomeIBGE': 'RIO DE JANEIRO',
    'pais': 'BRASIL',
    'uf':

In [20]:
mes_ini = '06'
ano_ini = 2018
mes_fim = '07'
ano_fim = 2018
pagina = 1
url = 'http://www.transparencia.gov.br/api-de-dados/cartoes?mesExtratoInicio={}%2F{}&mesExtratoFim={}%2F{}&pagina={}'
response = requests.get(url.format(mes_ini, ano_ini, mes_fim, ano_fim, pagina))
total = len(response.json())

while len(response.json()) > 0:
    pagina += 1
    print('Buscando página ', pagina, '...')
    response = requests.get(url.format(mes_ini, ano_ini, mes_fim, ano_fim, pagina))
    total += len(response.json())

print('Transações: ', total)

Buscando página  2 ...
Buscando página  3 ...
Buscando página  4 ...
Buscando página  5 ...
Buscando página  6 ...
Buscando página  7 ...
Buscando página  8 ...
Buscando página  9 ...
Buscando página  10 ...
Buscando página  11 ...
Buscando página  12 ...
Buscando página  13 ...
Buscando página  14 ...
Buscando página  15 ...
Buscando página  16 ...
Buscando página  17 ...
Buscando página  18 ...
Buscando página  19 ...
Buscando página  20 ...
Buscando página  21 ...
Buscando página  22 ...
Buscando página  23 ...
Buscando página  24 ...
Buscando página  25 ...
Buscando página  26 ...
Buscando página  27 ...
Buscando página  28 ...
Buscando página  29 ...
Buscando página  30 ...
Buscando página  31 ...
Buscando página  32 ...
Buscando página  33 ...
Buscando página  34 ...
Buscando página  35 ...
Buscando página  36 ...
Buscando página  37 ...
Buscando página  38 ...
Buscando página  39 ...
Buscando página  40 ...
Buscando página  41 ...
Buscando página  42 ...
Buscando página  43 ...


KeyboardInterrupt: 

In [21]:
url = 'https://api.exchangerate-api.com/v4/latest/BRL'
response = requests.get(url)
print(response.json())

{'base': 'BRL', 'date': '2019-11-18', 'time_last_updated': 1574035858, 'rates': {'BRL': 1, 'AED': 0.876259, 'ARS': 14.247208, 'AUD': 0.350556, 'BGN': 0.422652, 'BSD': 0.238629, 'CAD': 0.315568, 'CHF': 0.235981, 'CLP': 191.460194, 'CNY': 1.670972, 'COP': 831.913043, 'CZK': 5.530322, 'DKK': 1.615112, 'DOP': 12.621372, 'EGP': 3.837545, 'EUR': 0.216058, 'FJD': 0.522173, 'GBP': 0.184995, 'GTQ': 1.840339, 'HKD': 1.865387, 'HRK': 1.607555, 'HUF': 72.350504, 'IDR': 3341.616612, 'ILS': 0.829082, 'INR': 17.100065, 'ISK': 29.370283, 'JPY': 25.908625, 'KRW': 277.572719, 'KZT': 92.883495, 'MXN': 4.589124, 'MYR': 0.989842, 'NOK': 2.171787, 'NZD': 0.372941, 'PAB': 0.238629, 'PEN': 0.804332, 'PHP': 12.070536, 'PKR': 36.867052, 'PLN': 0.925106, 'PYG': 1594.5, 'RON': 1.030664, 'RUB': 15.216797, 'SAR': 0.893994, 'SEK': 2.302407, 'SGD': 0.324473, 'THB': 7.2058, 'TRY': 1.370726, 'TWD': 7.274742, 'UAH': 5.764056, 'USD': 0.238351, 'UYU': 8.983099, 'VND': 5531.578947, 'ZAR': 3.514419}}


In [23]:
from datetime import datetime
timestamp = 1574035858
dt_object = datetime.fromtimestamp(timestamp)

In [24]:
dt_object

datetime.datetime(2019, 11, 17, 21, 10, 58)

In [34]:
'{0:%d/%m/%Y %H:%M:%S}'.format(dt_object)

'17/11/2019 21:10:58'

In [35]:
'{0:%d/%B/%Y %H:%M:%S}'.format(dt_object)

'17/November/2019 21:10:58'

In [38]:
age = 10
f'{age}'

'10'